In [ ]:
from google.colab import userdata
API_KEY = userdata.get('GoogleAI_API_KEY')

In [ ]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
genai.configure(api_key=API_KEY)
gemini = genai.GenerativeModel(model_name="models/gemini-2.0-flash-exp")

In [ ]:
try:
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", google_api_key=API_KEY)
except Exception as e:
    print(e)

In [ ]:
res = gemini.generate_content('who are you?')

In [ ]:
print(res)

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader("dataset.csv")

data = loader.load()

In [ ]:
data[:5]

In [ ]:
for doc in data[:5]:  # Check the first 5 entries
    print(doc.page_content)

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

texts = [doc.page_content for doc in data]

# Generate embeddings
embeddings = embedder.encode(texts, convert_to_tensor=False)  # FAISS works with numpy arrays
print(f"Embeddings shape: {embeddings.shape}")

In [ ]:
embeddings[:4]

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings

# Create a FAISS vector store from your documents and embeddings
# Use SentenceTransformerEmbeddings to wrap your SentenceTransformer model
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(
    documents=data,
    embedding=embeddings
)

# Save the index (optional, for reuse)
vector_store.save_local("faiss_index")

In [ ]:
print(llm.invoke("Hello, are you working?"))

In [ ]:
from langchain.chains import RetrievalQA

# Set up the retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 3})  # Retrieve top 3 matches

# Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Combines retrieved docs into the prompt
    retriever=retriever,
    return_source_documents=True  # Optional: see what was retrieved
)

# Test it
query = "what best products can men use and suggest me products to handle acne scars?"
result = qa_chain({"query": query})
print(result["result"])
print(result["source_documents"])